In [194]:
import requests
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

import pandas as pd
from random import randint

# Web scraping fake news headlines related to covid-19

The ultimate goal is to build an intelligent system which can automatically montor, flag and track trends in COVAX fake news. To do so, we need to find text data related to fake and real news. As a starting point I am checking Poynter - https://www.poynter.org/. Poynter has a list of over 10,000 fake afticles which they have fact checked. The problem is, this data exists on their webiste and is not downloadable. Therefore we need to scrape their website. Their data is split between 78 pages which need to be clicked through, so when scraping, we need to loop through these pages. Since we want to train a model specifically for vaccines, we will filter the URL query for vaccines only. We could scrape their entire website for covid missinformation, however we also need to find a similar number of true articles for training the machine learning model.

In [195]:
headers = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.192 Safari/537.36'

label = []
headline = []

for i in range(1, 79):
    URL = 'https://www.poynter.org/ifcn-covid-19-misinformation/page/{}/?search_terms=vaccine'.format(i)
    page = requests.get(URL, headers={'User-Agent': headers})
    soup = BeautifulSoup(page.content, 'html.parser')
    elems = soup.find_all('h2', class_="entry-title")

    for header in elems:
        label.append(header.text.split(':')[0].strip("\n").strip())
        headline.append(header.text.split(':')[1].strip("\n").strip())
    if i % 3 == 0:
        print("Page number {} of 78 scraped".format(i))

Page number 3 of 78 scraped
Page number 6 of 78 scraped
Page number 9 of 78 scraped
Page number 12 of 78 scraped
Page number 15 of 78 scraped
Page number 18 of 78 scraped
Page number 21 of 78 scraped
Page number 24 of 78 scraped
Page number 27 of 78 scraped
Page number 30 of 78 scraped
Page number 33 of 78 scraped
Page number 36 of 78 scraped
Page number 39 of 78 scraped
Page number 42 of 78 scraped
Page number 45 of 78 scraped
Page number 48 of 78 scraped
Page number 51 of 78 scraped
Page number 54 of 78 scraped
Page number 57 of 78 scraped
Page number 60 of 78 scraped
Page number 63 of 78 scraped
Page number 66 of 78 scraped
Page number 69 of 78 scraped
Page number 72 of 78 scraped
Page number 75 of 78 scraped
Page number 78 of 78 scraped


In [196]:
df = pd.DataFrame(list(zip(label, headline)), 
               columns =['label', 'headline']) 
df = df.drop_duplicates(subset=['headline'])

keep_labels = ['FALSE', 'False', 
               'Misleading', 'MISLEADING', 
               'No evidence', 'Missing context', 
               'Partially false', 'misleading ', 
               'mostly false', 'NO EVIDENCE',
               'Mostly false', 'Manipulated', 
               'Misleading/False', 'Not true', 
               'Mainly false', ]

df = df[df['label'].isin(keep_labels)].reset_index(drop=True)

In [197]:
df.shape

(1119, 2)

We have 1119 news headlines related to fake vaccine news. Below, we print 5 at random:

In [198]:
x = [randint(0, len(df)) for x in range(5)]

for i in x:
    print("Headline: {}".format(df.loc[i]['headline']))
    print("Label: {}\n".format(df.loc[i]['label']))

Headline: The measles-mumps-rubella vaccine reduces the severity of COVID-19.
Label: No evidence

Headline: Vaccines killed 181 people in the United States
Label: FALSE

Headline: We do not need a vaccine. We need a body that can resist this and any other virus
Label: FALSE

Headline: A sharp increase in COVID-19 cases in England and Israel after the rollout of the vaccines proves the inefficacy of the vaccine
Label: MISLEADING

Headline: India is suing Microsoft founder Bill Gates because of his vaccine against coronavirus that killed 77,000 girls.
Label: FALSE

